<a href="https://colab.research.google.com/github/sergmasl/a_b_tests_scripts/blob/main/a_b_test_sample_size_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This cell initializes the Google Cloud BigQuery client, authenticates the Colab environment,
# and sets the project ID for subsequent BigQuery operations.
# Install the Google Cloud BigQuery client library if not already installed
!pip install google-cloud-bigquery

# Import necessary libraries
from google.cloud import bigquery
from google.colab import auth

# Authenticate your Colab environment
# This will prompt you to log in to your Google account and grant permissions.
print("Authenticating user...")
auth.authenticate_user()
print("User authenticated.")

# Define the BigQuery project ID
project_id = 'endel-analytics'

# Create a BigQuery client for the specified project
client = bigquery.Client(project=project_id)

print(f"Successfully connected to BigQuery project: {client.project}")

Authenticating user...
User authenticated.
Successfully connected to BigQuery project: endel-analytics


In [ ]:
# This cell constructs and executes a BigQuery SQL query to fetch A/B test data
# from the specified tables and loads the results into a Pandas DataFrame named `df`.
import pandas as pd

# Construct the SQL query
# Use CURRENT_DATE() to get the current date and DATE_SUB to subtract 10 days
query = f"""
WITH
table_labels as (
SELECT DISTINCT canonical_user_id as user_id,  exp_flowScreensSlot20 as label
FROM `endel-analytics.endel_data.v5_a_b_labels_new_users`
WHERE event_date BETWEEN '2026-01-08' AND '2026-01-20'
AND exp_flowScreensSlot20 IN ('"Catalog Fragments: text2sound Freemium [A]"', '"Catalog Fragments: text2sound Freemium [B]"')
)
SELECT table_labels.label, table_labels.user_id, user_info.conversion_day0,
user_info.conversion_day7, user_info.conversion_rcat_trial_started_day0,
user_info.arpu_day0, user_info.arpu_day7,
user_info.retention_day1, user_info.retention_day7,
FROM table_labels LEFT JOIN `endel-analytics.endel_data.v5_user_info` user_info
ON table_labels.user_id = user_info.user_id
AND user_creation_date BETWEEN '2026-01-08' AND '2026-01-20'
"""

print(f"Executing BigQuery query")

# Run the query and convert results to a pandas DataFrame
try:
    query_job = client.query(query)
    df = query_job.to_dataframe()

    print("Query successful")

except Exception as e:
    print(f"Error executing BigQuery query: {e}")
    print("Please ensure the table path is correct and your account has appropriate BigQuery permissions (e.g., bigquery.dataViewer on the dataset).")